# Generative Adversarial Network (GAN)

## Concept

GAN is composed of two neural networks that compete with each other, hence the name adversarial. There's the **Generator** network (G) and **Discriminator** network (D).

### Generator Network

The generator's task is to generate _fake data_ that could, hopefully, fool the discriminator. The generator does not have access to the _real data_ that the discriminator expects, instead it improves its ability to generate fake data by learning what feature of the data is expected by the discriminator.

- Input: vector of **noise**
- Output: fake data with the **same shape** as the real data

Stochastic gradient for generator network:

<center> $\bigtriangledown_{\theta_{d}}\frac{1}{m} \displaystyle\sum_{i=1}^{m}log (1-D(G(z^{(i)})))$ </center>

The target is to **minimize** the value of $1-D(G(z^{(i)}))$ which means **maximizing** the value of $D(G(z^{(i)})$, the probability that the discriminator passes the fake data generated by the generator as real.

### Discriminator Network

The discriminator's task is to determine if a data is _real_ or _fake_. The discriminator is actually updated by **ascending** its stochastic gradient, instead of descending.

- Input: real and fake **data**
- Output: binary (is the data real or fake)

Stochastic gradient for discriminator network:

<center> $\bigtriangledown_{\theta_{d}}\frac{1}{m} \displaystyle\sum_{i=1}^{m}[log D(x^{(i)}) + log (1-D(G(z^{(i)})))]$ </center>

## GAN for MNIST Dataset with Keras

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import sys

from keras.datasets import mnist
from keras.layers import Input, Dense, Reshape, Flatten, Dropout
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.models import load_model
from keras.optimizers import Adam

class GAN():
    
    def __init__(self):
        self.img_rows = 28 
        self.img_cols = 28
        self.channels = 1
        self.img_shape = (self.img_rows, self.img_cols, self.channels)
        
        self.noise_shape = (100,)

        # Manually tune down learning rate to avoid oscillation
        optimizer = Adam(lr=0.0002, beta_1=0.5)

        # -------------
        # Discriminator
        # -------------
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(loss='mean_squared_error', 
                                   optimizer=optimizer,
                                   metrics=['accuracy'])
        # ---------
        # Generator
        # ---------
        self.generator = self.build_generator()
        self.generator.compile(loss='mean_squared_error',
                               optimizer=optimizer)
        # --------
        # Combined
        # --------
        # The combined model is created by stacking generator and discriminator.
        # Noise ---Generator--> Generated Image ---Discriminator--> Validity
        
        z = Input(shape=self.noise_shape)
        img = self.generator(z)

        # For the combined model we will only train the generator
        self.discriminator.trainable = False
        
        validity = self.discriminator(img)

        self.combined = Model(z, validity)
        self.combined.compile(loss='mean_squared_error',
                              optimizer=optimizer)

        
    def save_models(self, epoch=None):
        if epoch is None:
            self.discriminator.save('gan/models/mnist-discriminator.h5')
            self.generator.save('gan/models/mnist-generator.h5')
            self.combined.save('gan/models/mnist-combined.h5')
        else:
            self.discriminator.save('gan/models/mnist-discriminator-{}.h5'.format(int(epoch)))
            self.generator.save('gan/models/mnist-generator-{}.h5'.format(int(epoch)))
            self.combined.save('gan/models/mnist-combined-{}.h5'.format(int(epoch)))
        
        
    def load_models(self, epoch=None): 
        if epoch is None:
            self.discriminator = load_model('gan/models/mnist-discriminator.h5')
            self.generator = load_model('gan/models/mnist-generator.h5')
            self.combined = load_model('gan/models/mnist-combined.h5')
        else:
            self.discriminator = load_model('gan/models/mnist-discriminator-{}.h5'.format(int(epoch)))
            self.generator = load_model('gan/models/mnist-generator-{}.h5'.format(int(epoch)))
            self.combined = load_model('gan/models/mnist-combined-{}.h5'.format(int(epoch)))
            
        
    def build_discriminator(self):
        model = Sequential()
        model.add(Flatten(input_shape=self.img_shape))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(256))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(1, activation='sigmoid'))
        
        model.summary()

        # Discriminator takes an image as an input and outputs its validity
        img = Input(shape=self.img_shape)
        validity = model(img)

        return Model(img, validity)

        
    def build_generator(self):
        # BatchNormalization maintains the mean activation close to 0
        # and the activation standard deviation close to 1
        model = Sequential()
        model.add(Dense(256, input_shape=self.noise_shape))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(1024))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(np.prod(self.img_shape), activation='sigmoid'))
        model.add(Reshape(self.img_shape))
        
        model.summary()

        # Generator takes noise as an input and outputs an image
        noise = Input(shape=self.noise_shape)
        img = model(noise)

        return Model(noise, img)
    
    
    def train(self, epochs, batch_size=128, export_image_interval=250, save_model_interval=2500):

        # Load the dataset
        (X_train, _), (_, _) = mnist.load_data()
        
        # X_train shape: (m, W, H)
        
        # Rescale 0 to 1
        X_train = (X_train.astype(np.float32)) / 255
        
        # Change X_train's shape to (m, W, H, c)
        X_train = np.expand_dims(X_train, axis=3)

        half_batch = int(batch_size / 2)

        for epoch in range(epochs):

            # -------------------
            # Train Discriminator
            # -------------------
            # Select a random half batch of images
            idx = np.random.randint(0, X_train.shape[0], half_batch)
            imgs = X_train[idx]

            # Generate a half batch of new images
            noise = np.random.normal(0, 1, (half_batch, 100))
            gen_imgs = self.generator.predict(noise)

            # Train the discriminator
            d_loss_real = self.discriminator.train_on_batch(imgs, np.ones((half_batch, 1)))
            d_loss_fake = self.discriminator.train_on_batch(gen_imgs, np.zeros((half_batch, 1)))
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

            # ---------------
            # Train Generator
            # ---------------
            noise = np.random.normal(0, 1, (batch_size, 100))

            # The generator wants to fool the discriminator, hence trained with valid label (1)
            valid_y = np.array([1] * batch_size)

            # Train the generator
            g_loss = self.combined.train_on_batch(noise, valid_y)


            # Print progress
            print ("{:5d} [D loss: {}, acc_real: {:2f}, acc_fake: {:2f}] [G loss: {}]".format(epoch, d_loss[0], 100*d_loss_real[1], 100*d_loss_fake[1], g_loss))

            # Save generated samples at export_image_interval
            if epoch % export_image_interval == 0:
                r, c = 3, 3

                fig, axs = plt.subplots(r, c)
                
                prediction = self.discriminator.predict(gen_imgs)
                prediction = np.round(prediction)
                
                fooled_indices = []
                
                for i in range(len(prediction)):
                    if prediction[i] == 1:
                        fooled_indices.append(i)
                
                fooled_count = len(fooled_indices)
                
                # Export generated images that the discriminator classified as real
                cnt = 0
                if (fooled_count > 0):
                    for i in range(r):
                        for j in range(c):
                            # Prevent error where count exceeds the number of failed test case for discriminator
                            if (cnt >= fooled_count):
                                break
                            axs[i,j].imshow(gen_imgs[fooled_indices[cnt], :,:,0], cmap='gray')
                            axs[i,j].axis('off')
                            cnt += 1
                            
                fig.savefig("gan/images/mnist_{}_fooled.png".format(epoch))
                plt.close()
                
                # Export generated images that the discriminator classified as fake
                fig, axs = plt.subplots(r, c)
                success_count = len(prediction) - fooled_count
                cnt = 0
                if (success_count > 0):
                    for i in range(r):
                        for j in range(c):
                            # Find index not in fooled_indices
                            while cnt in fooled_indices:
                                cnt += 1
                                
                            # Prevent error where count exceeds the number of successful test case for discriminator
                            if (cnt >= success_count):
                                break
                            
                            axs[i,j].imshow(gen_imgs[cnt, :,:,0], cmap='gray')
                            axs[i,j].axis('off')
                            cnt += 1
                fig.savefig("gan/images/mnist_{}_detected.png".format(epoch))
                plt.close()
            
            
            # Save models at save_interval
            if epoch != 0 and epoch % save_model_interval == 0:
                self.save_models(epoch)
                


if __name__ == '__main__':
    gan = GAN()
    gan.train(epochs=30001, batch_size=64, export_image_interval=250, save_model_interval=2500)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_2 (Flatten)          (None, 784)               0         
_________________________________________________________________
dense_8 (Dense)              (None, 512)               401920    
_________________________________________________________________
leaky_re_lu_6 (LeakyReLU)    (None, 512)               0         
_________________________________________________________________
dense_9 (Dense)              (None, 256)               131328    
_________________________________________________________________
leaky_re_lu_7 (LeakyReLU)    (None, 256)               0         
_________________________________________________________________
dense_10 (Dense)             (None, 1)                 257       
Total params: 533,505
Trainable params: 533,505
Non-trainable params: 0
_________________________________________________________________
____

/home/rionaldichandraseta/miniconda3/envs/gan-image/lib/python3.6/site-packages/keras/engine/training.py:478: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


    0 [D loss: 0.31062832474708557, acc_real: 15.625000, acc_fake: 0.000000] [G loss: 0.4728468656539917]
    1 [D loss: 0.15114149451255798, acc_real: 62.500000, acc_fake: 100.000000] [G loss: 0.7265317440032959]
    2 [D loss: 0.11823553591966629, acc_real: 81.250000, acc_fake: 100.000000] [G loss: 0.7931737899780273]
    3 [D loss: 0.09703528881072998, acc_real: 90.625000, acc_fake: 100.000000] [G loss: 0.7981671094894409]
    4 [D loss: 0.07758110761642456, acc_real: 93.750000, acc_fake: 100.000000] [G loss: 0.8014616370201111]
    5 [D loss: 0.05352252721786499, acc_real: 100.000000, acc_fake: 100.000000] [G loss: 0.8228565454483032]
    6 [D loss: 0.04340119659900665, acc_real: 100.000000, acc_fake: 100.000000] [G loss: 0.8429672122001648]
    7 [D loss: 0.039036303758621216, acc_real: 100.000000, acc_fake: 100.000000] [G loss: 0.8652445077896118]
    8 [D loss: 0.027881525456905365, acc_real: 100.000000, acc_fake: 100.000000] [G loss: 0.8825604915618896]
    9 [D loss: 0.0262091

   76 [D loss: 0.024677865207195282, acc_real: 93.750000, acc_fake: 100.000000] [G loss: 0.9657765626907349]
   77 [D loss: 0.007740328088402748, acc_real: 100.000000, acc_fake: 100.000000] [G loss: 0.9290899038314819]
   78 [D loss: 0.023379303514957428, acc_real: 100.000000, acc_fake: 90.625000] [G loss: 0.9126975536346436]
   79 [D loss: 0.045417316257953644, acc_real: 100.000000, acc_fake: 93.750000] [G loss: 0.961456298828125]
   80 [D loss: 0.02746930718421936, acc_real: 96.875000, acc_fake: 100.000000] [G loss: 0.949923038482666]
   81 [D loss: 0.09400427341461182, acc_real: 93.750000, acc_fake: 78.125000] [G loss: 0.9515145421028137]
   82 [D loss: 0.02485497295856476, acc_real: 100.000000, acc_fake: 100.000000] [G loss: 0.9434548020362854]
   83 [D loss: 0.08704395592212677, acc_real: 96.875000, acc_fake: 84.375000] [G loss: 0.9578325152397156]
   84 [D loss: 0.0470290333032608, acc_real: 100.000000, acc_fake: 93.750000] [G loss: 0.9440213441848755]
   85 [D loss: 0.0478378832

  153 [D loss: 0.05800451710820198, acc_real: 96.875000, acc_fake: 87.500000] [G loss: 0.8732512593269348]
  154 [D loss: 0.11227734386920929, acc_real: 93.750000, acc_fake: 75.000000] [G loss: 0.8802850246429443]
  155 [D loss: 0.08346255123615265, acc_real: 93.750000, acc_fake: 90.625000] [G loss: 0.8130877017974854]
  156 [D loss: 0.0872763842344284, acc_real: 100.000000, acc_fake: 71.875000] [G loss: 0.9059334993362427]
  157 [D loss: 0.06313180178403854, acc_real: 93.750000, acc_fake: 93.750000] [G loss: 0.8315093517303467]
  158 [D loss: 0.09680815041065216, acc_real: 100.000000, acc_fake: 71.875000] [G loss: 0.824164092540741]
  159 [D loss: 0.04566976800560951, acc_real: 100.000000, acc_fake: 93.750000] [G loss: 0.8887379169464111]
  160 [D loss: 0.06190688535571098, acc_real: 96.875000, acc_fake: 93.750000] [G loss: 0.8827150464057922]
  161 [D loss: 0.06874323636293411, acc_real: 93.750000, acc_fake: 96.875000] [G loss: 0.8639883995056152]
  162 [D loss: 0.039357222616672516,

...

29665 [D loss: 0.21633976697921753, acc_real: 65.625000, acc_fake: 75.000000] [G loss: 0.35980409383773804]
29666 [D loss: 0.19320210814476013, acc_real: 56.250000, acc_fake: 78.125000] [G loss: 0.3857376277446747]
29667 [D loss: 0.22733205556869507, acc_real: 59.375000, acc_fake: 65.625000] [G loss: 0.3703932464122772]
29668 [D loss: 0.22731561958789825, acc_real: 62.500000, acc_fake: 56.250000] [G loss: 0.3845641016960144]
29669 [D loss: 0.18939554691314697, acc_real: 71.875000, acc_fake: 78.125000] [G loss: 0.3628106415271759]
29670 [D loss: 0.20553666353225708, acc_real: 62.500000, acc_fake: 78.125000] [G loss: 0.39384984970092773]
29671 [D loss: 0.2286890149116516, acc_real: 50.000000, acc_fake: 75.000000] [G loss: 0.3575555384159088]
29672 [D loss: 0.2253812998533249, acc_real: 56.250000, acc_fake: 56.250000] [G loss: 0.3780372738838196]
29673 [D loss: 0.24163448810577393, acc_real: 43.750000, acc_fake: 68.750000] [G loss: 0.371097207069397]
29674 [D loss: 0.21678805351257324, ac

29745 [D loss: 0.22270795702934265, acc_real: 50.000000, acc_fake: 71.875000] [G loss: 0.36363670229911804]
29746 [D loss: 0.20557086169719696, acc_real: 62.500000, acc_fake: 68.750000] [G loss: 0.3842543363571167]
29747 [D loss: 0.20881684124469757, acc_real: 53.125000, acc_fake: 78.125000] [G loss: 0.3665209114551544]
29748 [D loss: 0.20429010689258575, acc_real: 62.500000, acc_fake: 68.750000] [G loss: 0.33604344725608826]
29749 [D loss: 0.21962115168571472, acc_real: 62.500000, acc_fake: 68.750000] [G loss: 0.36254918575286865]
29750 [D loss: 0.1978611946105957, acc_real: 68.750000, acc_fake: 65.625000] [G loss: 0.327797532081604]
29751 [D loss: 0.21652871370315552, acc_real: 59.375000, acc_fake: 75.000000] [G loss: 0.35227078199386597]
29752 [D loss: 0.22911939024925232, acc_real: 59.375000, acc_fake: 71.875000] [G loss: 0.37086597084999084]
29753 [D loss: 0.2242431491613388, acc_real: 46.875000, acc_fake: 71.875000] [G loss: 0.3775365948677063]
29754 [D loss: 0.21777713298797607,

29823 [D loss: 0.19429129362106323, acc_real: 68.750000, acc_fake: 75.000000] [G loss: 0.3548330068588257]
29824 [D loss: 0.21286389231681824, acc_real: 62.500000, acc_fake: 65.625000] [G loss: 0.3979244828224182]
29825 [D loss: 0.20706996321678162, acc_real: 65.625000, acc_fake: 78.125000] [G loss: 0.3335747718811035]
29826 [D loss: 0.23959176242351532, acc_real: 59.375000, acc_fake: 50.000000] [G loss: 0.37024638056755066]
29827 [D loss: 0.1910431683063507, acc_real: 59.375000, acc_fake: 87.500000] [G loss: 0.3973616361618042]
29828 [D loss: 0.21896356344223022, acc_real: 65.625000, acc_fake: 68.750000] [G loss: 0.4071398377418518]
29829 [D loss: 0.24146409332752228, acc_real: 46.875000, acc_fake: 65.625000] [G loss: 0.33753153681755066]
29830 [D loss: 0.21872511506080627, acc_real: 71.875000, acc_fake: 53.125000] [G loss: 0.3512466251850128]
29831 [D loss: 0.22443696856498718, acc_real: 62.500000, acc_fake: 65.625000] [G loss: 0.40154823660850525]
29832 [D loss: 0.2156425565481186, 

29901 [D loss: 0.22392389178276062, acc_real: 71.875000, acc_fake: 65.625000] [G loss: 0.39184796810150146]
29902 [D loss: 0.18843670189380646, acc_real: 68.750000, acc_fake: 84.375000] [G loss: 0.3957001268863678]
29903 [D loss: 0.21258701384067535, acc_real: 59.375000, acc_fake: 81.250000] [G loss: 0.3916967511177063]
29904 [D loss: 0.22351299226284027, acc_real: 65.625000, acc_fake: 68.750000] [G loss: 0.4197326898574829]
29905 [D loss: 0.19380217790603638, acc_real: 50.000000, acc_fake: 75.000000] [G loss: 0.41854149103164673]
29906 [D loss: 0.19277828931808472, acc_real: 75.000000, acc_fake: 59.375000] [G loss: 0.4065437912940979]
29907 [D loss: 0.19776339828968048, acc_real: 62.500000, acc_fake: 81.250000] [G loss: 0.3572055697441101]
29908 [D loss: 0.22448565065860748, acc_real: 65.625000, acc_fake: 46.875000] [G loss: 0.3883141875267029]
29909 [D loss: 0.17509736120700836, acc_real: 78.125000, acc_fake: 75.000000] [G loss: 0.3624994158744812]
29910 [D loss: 0.21507957577705383,

29980 [D loss: 0.21996082365512848, acc_real: 50.000000, acc_fake: 81.250000] [G loss: 0.34511417150497437]
29981 [D loss: 0.2097928822040558, acc_real: 62.500000, acc_fake: 71.875000] [G loss: 0.360107958316803]
29982 [D loss: 0.24875546991825104, acc_real: 65.625000, acc_fake: 59.375000] [G loss: 0.3602556884288788]
29983 [D loss: 0.237725168466568, acc_real: 59.375000, acc_fake: 56.250000] [G loss: 0.3685554265975952]
29984 [D loss: 0.24152876436710358, acc_real: 56.250000, acc_fake: 59.375000] [G loss: 0.3791952431201935]
29985 [D loss: 0.20960880815982819, acc_real: 78.125000, acc_fake: 71.875000] [G loss: 0.3817216753959656]
29986 [D loss: 0.2075912356376648, acc_real: 62.500000, acc_fake: 68.750000] [G loss: 0.3857675790786743]
29987 [D loss: 0.21785300970077515, acc_real: 68.750000, acc_fake: 75.000000] [G loss: 0.36654216051101685]
29988 [D loss: 0.24683821201324463, acc_real: 46.875000, acc_fake: 56.250000] [G loss: 0.36023807525634766]
29989 [D loss: 0.20240317285060883, acc

The following code tests if the load model is working by loading the model and generate images from noise

In [ ]:
gan2 = GAN()
gan2.load_models(100)

noise = np.random.normal(0, 1, (9, 100))
gen_imgs = gan2.generator.predict(noise)

fig, axs = plt.subplots(3, 3)

count = 0
for i in range(3):
    for j in range(3):
        # Prevent error where count exceeds the number of failed test case for discriminator
        axs[i,j].imshow(gen_imgs[count, :,:,0], cmap='gray')
        axs[i,j].axis('off')
        count += 1